In [80]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
pd.set_option('display.max_colwidth', None)

### Multiple page scrapping - from Pitchfork

In [221]:
r4 = requests.get('https://pitchfork.com/features/lists-and-guides/9288-the-top-100-tracks-of-2013/')
r4.status_code

200

In [261]:
# Instantiate BeatifulSoup class
sp = BeautifulSoup(r4.content, 'html.parser')

In [259]:
#Get page starts
starts = range(1,11) # because, 10 items per page
list(starts)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [244]:
songs = sp.find_all('div', attrs={'class': "list-blurb__artist-work"})
for i in songs:
    print(i)

<div class="list-blurb__artist-work"><ul class="artist-links artist-list list-blurb__artists"><li><a href="/artists/30051-factory-floor/">Factory Floor</a></li></ul><h2 class="list-blurb__work-title">"Fall Back"</h2></div>
<div class="list-blurb__artist-work"><ul class="artist-links artist-list list-blurb__artists"><li><a href="/artists/5758-thee-oh-sees/">Thee Oh Sees</a></li></ul><h2 class="list-blurb__work-title">"Toe Cutter - Thumb Buster"</h2></div>
<div class="list-blurb__artist-work"><ul class="artist-links artist-list list-blurb__artists"><li><a href="/artists/31225-the-range/">The Range</a></li></ul><h2 class="list-blurb__work-title">"Metal Swing"</h2></div>
<div class="list-blurb__artist-work"><ul class="artist-links artist-list list-blurb__artists"><li><a href="/artists/30523-foxygen/">Foxygen</a></li></ul><h2 class="list-blurb__work-title">"No Destruction"</h2></div>
<div class="list-blurb__artist-work"><ul class="artist-list list-blurb__artists"><li>Ty Segall</li></ul><h2 

In [337]:
songs[0].find('a').get('href')

'/artists/29170-autre-ne-veut/'

In [348]:
def get_song_info(songs): 
    titles_ = [ ]
    song_artists = [ ]
    song_links = [ ]

    for song in songs:
        title = song.find('ul').find('li').get_text() if song.find('ul').find('li') else 'Not informed.'
        song_artist = song.find('h2').get_text(strip = True).strip('"') if song.find('h2') else 'Not informed.'
        song_link = 'https://pitchfork.com' + song.find('a').get('href') if song.find('a') else "No Link available"


        titles_.append(title)
        song_artists.append(song_artist)
        song_links.append(song_link)

    return {'Title': titles_,'Artist': song_artists, 'Link':song_links}

In [349]:
song_dict = get_song_info(songs)
song_dict

{'Title': ['Autre Ne Veut',
  'Deafheaven',
  'Daft Punk',
  'Disclosure',
  'Haim',
  'Ciara',
  'Arcade Fire',
  'Vampire Weekend',
  'Kanye West',
  'Drake'],
 'Artist': ['Play by Play',
  'Dreamhouse',
  'Get Lucky" [ft. Pharrell]',
  'Latch',
  'The Wire',
  'Body Party',
  'Reflektor',
  'Hannah Hunt',
  'New Slaves',
  'Hold On, We\'re Going Home" [ft. Majid Jordan]'],
 'Link': ['https://pitchfork.com/artists/29170-autre-ne-veut/',
  'https://pitchfork.com/artists/30011-deafheaven/',
  'https://pitchfork.com/artists/990-daft-punk/',
  'No Link available',
  'No Link available',
  'No Link available',
  'https://pitchfork.com/artists/221-arcade-fire/',
  'No Link available',
  'No Link available',
  'No Link available']}

In [350]:
song_df = pd.DataFrame()

for start in starts:
    r = requests.get(f'https://pitchfork.com/features/lists-and-guides/9288-the-top-100-tracks-of-2013/?page={start}')
    sp = BeautifulSoup(r.content, 'html.parser')
    songs = sp.find_all('div', attrs={'class': "list-blurb__artist-work"})
    info_dct = get_song_info(songs)
    new_df = pd.DataFrame.from_dict(info_dct)
    song_df = pd.concat([song_df, new_df])
    
song_df
    

,Title,Artist,Link
0,Factory Floor,Fall Back,https://pitchfork.com/artists/30051-factory-floor/
1,Thee Oh Sees,Toe Cutter - Thumb Buster,https://pitchfork.com/artists/5758-thee-oh-sees/
2,The Range,Metal Swing,https://pitchfork.com/artists/31225-the-range/
3,Foxygen,No Destruction,https://pitchfork.com/artists/30523-foxygen/
4,Ty Segall,Sleeper,No Link available
...,...,...,...
5,Ciara,Body Party,No Link available
6,Arcade Fire,Reflektor,https://pitchfork.com/artists/221-arcade-fire/
7,Vampire Weekend,Hannah Hunt,No Link available
8,Kanye West,New Slaves,No Link available


#### From Wikipedia

In [81]:
r = requests.get('https://en.wikipedia.org/wiki/Lists_of_songs')
r.status_code

200

In [82]:
soup = BeautifulSoup(r.content, 'html.parser')

In [83]:
all_songs = soup.find('div', attrs={'class': "div-col"})
lists = all_songs.find_all('li')
titles = []
links = []
for i in lists:
    titles.append(i.get_text())
    links.append('https://en.wikipedia.org' + i.find('a').get('href'))

    
dct = {'Title': titles,'Link': links}

In [84]:
df_list = pd.DataFrame(dct)
df_list

,Title,Link
0,List of songs recorded by Arijit Singh,https://en.wikipedia.org/wiki/List_of_songs_recorded_by_Arijit_Singh
1,List of songs recorded by 2PM,https://en.wikipedia.org/wiki/List_of_songs_recorded_by_2PM
2,List of songs recorded by 311,https://en.wikipedia.org/wiki/List_of_songs_recorded_by_311
3,List of songs recorded by 4Minute,https://en.wikipedia.org/wiki/List_of_songs_recorded_by_4Minute
4,List of songs recorded by 911,https://en.wikipedia.org/wiki/List_of_songs_recorded_by_911
...,...,...
294,List of songs recorded by Wolf Alice,https://en.wikipedia.org/wiki/List_of_songs_recorded_by_Wolf_Alice
295,List of songs recorded by Wolfmother,https://en.wikipedia.org/wiki/List_of_songs_recorded_by_Wolfmother
296,List of songs recorded by Alka Yagnik,https://en.wikipedia.org/wiki/List_of_songs_recorded_by_Alka_Yagnik
297,"List of songs recorded by ""Weird Al"" Yankovic",https://en.wikipedia.org/wiki/List_of_songs_recorded_by_%22Weird_Al%22_Yankovic


#### From Rolling Stone

In [104]:
r2 = requests.get('https://www.rollingstone.com/music/music-lists/')
r2.status_code

200

In [143]:
soup_2 = BeautifulSoup(r2.content, 'html.parser')

In [144]:
new_songs_list = soup_2.find('ul', attrs={'class': "l-river"})
new_songs_list#.descendants

<ul class="l-river">
<li class="l-river__item">
<article class="c-card c-card--domino">
<a class="c-card__wrap" href="https://www.rollingstone.com/music/music-lists/best-billie-eilish-songs-1289703/">
<figure class="c-card__image">
<div class="c-crop c-crop--size-domino">
<noscript><img alt="billie eilish best songs" class="c-crop__img wp-post-image" data-src="https://www.rollingstone.com/wp-content/uploads/2022/01/billie-eilish-silo.jpg?resize=940,705&amp;w=940" height="705" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==" width="940"/></noscript> </div><!-- .c-crop -->
</figure><!-- .c-card__image -->
<header class="c-card__header">
<div class="c-badge c-badge--sponsored">
</div><!-- .c-badge--sponsored -->
<h3 class="c-card__heading t-bold">
	
	The 20 Best Billie Eilish Songs</h3><!-- .c-card__heading -->
<div class="c-card__tag t-bold t-bold--upper">
<span class="screen-reader-text">Posted in:</span>
<span class="c-card__featured-tag"> Music Lists</s

In [178]:

lists_2 = new_songs_list.find_all('li', attrs = {'class': "l-river__item"} )
gp_titles = []
music_lists_link = []
for i in lists_2:
    #print(i)
    link = i.find('a', attrs = {'class':"c-card__wrap"})
    if link and link.get('href') :
        music_lists_link.append(link.get('href'))
        gp_titles.append(link.find('h3').get_text(strip=True))
    
    dictn = {'Group Titles': gp_titles,'Group Link': music_lists_link}

In [179]:
df_lists = pd.DataFrame(dictn)
df_lists

,Group Titles,Group Link
0,The 20 Best Billie Eilish Songs,https://www.rollingstone.com/music/music-lists/best-billie-eilish-songs-1289703/
1,The 40 Greatest Dr. Dre Productions,https://www.rollingstone.com/music/music-lists/best-dr-dre-produced-songs-1289410/
2,The 25 Most Stylish Musicians Now,https://www.rollingstone.com/music/music-lists/25-most-stylish-musicians-now-1282425/
3,The Most Anticipated Hip-Hop Albums of 2022,https://www.rollingstone.com/music/music-lists/2022-most-anticipated-hip-hop-albums-1282329/
4,Nine of the 10 Highest-Paid Musicians of 2021 Were Men,https://www.rollingstone.com/music/music-lists/highest-paid-musicians-2021-bruce-springsteen-jay-z-taylor-swift-1281654/
5,Black Thought: My Life in 20 Songs,https://www.rollingstone.com/music/music-lists/roots-black-thought-songs-1107397/
6,The 10 Best K-Pop Debuts of 2021,https://www.rollingstone.com/music/music-lists/the-10-best-k-pop-debuts-of-2021-1277346/
7,The Best Music Books of 2021,https://www.rollingstone.com/music/music-lists/the-best-music-books-of-2021-1274301/
8,Rob Sheffield’s Top 25 Songs of 2021,https://www.rollingstone.com/music/music-lists/best-songs-2021-rob-sheffield-1268531/
9,Rob Sheffield’s Top 20 Albums of 2021,https://www.rollingstone.com/music/music-lists/rob-sheffield-best-albums-of-2021-1268527/


####  Something to try my fingers from spinditty.com 🤷‍♀️😁

In [181]:
r3 = requests.get('https://spinditty.com/playlists/Best-Songs-About-Falling')
r3.content

b'<!DOCTYPE html><html id="phx-wrapper" lang="en"><head><title>100 Best Songs About Falling - Spinditty</title><link rel="canonical" href="https://spinditty.com/playlists/Best-Songs-About-Falling"><meta property="author" content="Ansel Pereira"><meta name="description" content="Rise up with an epic playlist of songs about falling."><meta name="keywords" content="Playlists"><meta name="section" content="Playlists"><meta name="pagetype" content="article"><meta name="companies" content="best-songs-about-falling; best-songs-about-falling; best-songs-about-falling"><meta name="robots" content="max-image-preview:large"><meta name="item-id" content="ci02993329c000278e"><meta name="published" content="2022-02-04T06:20:02Z"><meta name="dcterms.rightsHolder" content="Maven Media Brands, LLC and respective content providers on this website. Other product and company names shown may be trademarks of their respective owners. Maven Media Brands, LLC and respective content providers to this website m

In [182]:
soup_3 = BeautifulSoup(r3.content, 'html.parser')

In [199]:
top = soup_3.find('ol')
top_ = top.find_all('li')
top_
#top_100_a = 

[<li>“Falling”—Trevor Daniel</li>,
 <li>“Can’t Help Falling in Love”—Elvis Presley</li>,
 <li>“Falling”—Harry Styles</li>,
 <li>“Fall”—Eminem</li>,
 <li>“All Falls Down”—Alan Walker</li>,
 <li>“Fall in Line”—Christina Aguilera featuring Demi Lovato</li>,
 <li>“Falling Slowly”—Glen Hansard &amp; Marketa Irglova</li>,
 <li>“Fallin’ (Adrenaline)”—Why Don’t We</li>,
 <li>“Falling Like the Stars”—James Arthur</li>,
 <li>“Fall for You”—Secondhand Serenade</li>]

In [334]:
title_n_artist = [ ]
for t in top_:
    title_n_artist.append(t.get_text(strip=True).split('—'))


In [335]:
titles_ = [ ]
artists_ = [ ]
for i in title_n_artist:
    titles_.append(i[0])
    artists_.append(i[1])
    
    
    dictn_ = {'Titles': titles_,'Artists': artists_}

In [336]:
df_list2 = pd.DataFrame(dictn_)
df_list2

,Titles,Artists
0,“Falling”,Trevor Daniel
1,“Can’t Help Falling in Love”,Elvis Presley
2,“Falling”,Harry Styles
3,“Fall”,Eminem
4,“All Falls Down”,Alan Walker
5,“Fall in Line”,Christina Aguilera featuring Demi Lovato
6,“Falling Slowly”,Glen Hansard & Marketa Irglova
7,“Fallin’ (Adrenaline)”,Why Don’t We
8,“Falling Like the Stars”,James Arthur
9,“Fall for You”,Secondhand Serenade


### MVP 

In [263]:
# Get first song list from another notebook and take a peek
billboard100 = pd.read_csv('billboard100.tsv', sep='\t')
billboard100.head()

,Title,Artist
0,We Don't Talk About Bruno,"Carolina Gaitan, Mauro Castillo, Adassa, Rhenzy Feliz, Diane Guerrero, Stephanie Beatriz & Encanto Cast"
1,Easy On Me,Adele
2,Heat Waves,Glass Animals
3,Stay,The Kid LAROI & Justin Bieber
4,Super Gremlin,Kodak Black


In [322]:

#billboard100['combined']=billboard100['Title'].astype(str) + '—'+ billboard100['Artist']
billboard100 = billboard100.drop('combined', axis=1)

In [323]:
billboard100[['Title', 'Artist']].sample(n=1)

,Title,Artist
48,Light Switch,Charlie Puth


In [324]:
# Display first dataframe
display(billboard100)

,Title,Artist
0,We Don't Talk About Bruno,"Carolina Gaitan, Mauro Castillo, Adassa, Rhenzy Feliz, Diane Guerrero, Stephanie Beatriz & Encanto Cast"
1,Easy On Me,Adele
2,Heat Waves,Glass Animals
3,Stay,The Kid LAROI & Justin Bieber
4,Super Gremlin,Kodak Black
...,...,...
95,No Love,Summer Walker & SZA
96,Megan's Piano,Megan Thee Stallion
97,Eazy,The Game & Kanye West
98,Fish Scale,YoungBoy Never Broke Again


In [351]:
# Display second dataframe
display(song_df)

,Title,Artist,Link
0,Factory Floor,Fall Back,https://pitchfork.com/artists/30051-factory-floor/
1,Thee Oh Sees,Toe Cutter - Thumb Buster,https://pitchfork.com/artists/5758-thee-oh-sees/
2,The Range,Metal Swing,https://pitchfork.com/artists/31225-the-range/
3,Foxygen,No Destruction,https://pitchfork.com/artists/30523-foxygen/
4,Ty Segall,Sleeper,No Link available
...,...,...,...
5,Ciara,Body Party,No Link available
6,Arcade Fire,Reflektor,https://pitchfork.com/artists/221-arcade-fire/
7,Vampire Weekend,Hannah Hunt,No Link available
8,Kanye West,New Slaves,No Link available


In [353]:
# Define a MVP - a song recommender
def song_recommender(song_title, df1, df2):
    if song_title in df1.Title.unique():
        return df1[df1['Title'] != song_title].sample(n=1) # ['Title'].item()
    else:
        return df2[df2['Title'] != song_title].sample(n=1) # ['Title'].item()

In [355]:
# Get song title from user
song_title = input('Type the song title of a song: ')

Type the song title of a song: Vampire Weekend


In [356]:
# Use the mvp to recommend another song
output_song = song_recommender(song_title, billboard100, song_df)
print(output_song)

            Title         Artist  \
0  Phosphorescent  Song for Zula   

                                                 Link  
0  https://pitchfork.com/artists/3475-phosphorescent/  
